In [ ]:
!pip install OrderedSet
!pip install numpy
!pip install pandas
!pip install matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from orderedset import OrderedSet
from time import sleep
import sys

## Get transactions from 1 address
Seems like this API limits this to 10 000 txs.

In [ ]:
def get_transactions_from_address(address,start_block,end_block):
  URL = "https://api.etherscan.io/api"
  
  # parameters
  api_key = "8FHTNPR8GQPAEVSRHBVDZ4TWEJQCCUB1NX"
  # defining a params dict for the parameters to be sent to the API
  PARAMS = {'module':'account','action':'txlist','address':address,'startblock':start_block,'endblock':end_block,'apikey':api_key}
  
  # sending get request and saving the response as response object
  try:
    r = requests.get(url = URL, params = PARAMS)
  except Exception:
    # If connection error, skip this address
    print("[Error] Error connecting to API, skipping address...")
    return pd.DataFrame({"from":[],"to":[],"value":[]}),0  
  
  # extracting data in json format
  data = r.json()
  if data["status"] != "0":
    print("{}".format(data["message"]))

  if len(data["result"]) > 0:
      # Load response into Dataframe
      try:
        df = pd.DataFrame(data['result'])
      except Exception:
        print("Error reading dataFrame from values, written to error.txt")
        with open("error.txt",'w') as f:
          f.write(data["result"])
          return pd.DataFrame({"from":[],"to":[],"value":[]}),0

      # Throw away some weird values
      df = df.loc[df['isError'] == "0"]
      df = df.loc[df["value"] != "0"]

      full_txn_count = len(df)

      # df = df[["timeStamp","from","to","value","gas",'isError',"gasUsed"]]
      df = df[["from","to","value"]]

      # Convert unix timestamp into something readable
      # df['timeStamp'] = df['timeStamp'].apply(lambda ts: datetime.fromtimestamp(int(ts)).strftime("%c"))

      # Group txs by (from,to) and sum value
      df['value'] = df['value'].apply(lambda x: int(x))   # Convert value str to int
      df = df.groupby(['from', 'to']).agg({'value':'sum'})
      df = df.reset_index(level=df.index.names)

      if len(df) == 10000:
        print("[Warning] 10k txs")
      return df,full_txn_count
  
  return pd.DataFrame({"from":[],"to":[],"value":[]}),0

## Load addresses with highest balances
I copied these addresses manually btw, seems like there is no API call for it

In [ ]:
## Get addresses with highest balance
addresses = []
with open("top_1000_addresses.txt",'r') as f:
    addresses = list(f.read().split("\n"))

## BFS walk for 1 month period

In [ ]:
# Starting Jan 17, a snapshot each 8 months
# Block numbers from https://blockchair.com/ethereum/
snapshot_blocklimits = []
snapshot_blocklimits.append((2912407,3100153))    # Jan '17
snapshot_blocklimits.append((4467005,4652925))    # Nov '17
snapshot_blocklimits.append((6249399,6430272))    # Sep '18
snapshot_blocklimits.append((8062293,8261511))    # Jul '19
snapshot_blocklimits.append((9976964,10176689))    # May '20
snapshot_blocklimits.append((11948960,12150244))    # Mar '21

# Only collect 100 million txs
collect_max = 1e6
collected = 0

# Get start and end block
snapshot = 0
start_block = snapshot_blocklimits[snapshot][0]
end_block = snapshot_blocklimits[snapshot][1]

output_file = "Ntxs_bf_blocks_{}_{}.txt".format(start_block,end_block)
f = open(output_file,'w',buffering=1)

queue = OrderedSet(addresses[:100])
seen_addresses = set()

while len(queue) > 0 and collected < collect_max:
    address = queue.pop()
    
    df, n = get_transactions_from_address(address,start_block,end_block)
    collected += n # Count new txs

    # Add all unseen addresses to queue
    seen_addresses.add(address)
    new_addresses = set([x for x in list(df["from"])+list(df["to"]) if x not in seen_addresses])
    for new in new_addresses:
        if len(queue) < 10000:
            queue.add(new)

    # sleep(0.25) # Wait 0.20s because we are limited to 5tx/s

    # Append dataframe in CSV format to output file
    f.write(df.to_csv(header=False,index=False))
    print("Txs: {} of {}, New txs: {}, seen addr: {}, Q = {} ({} new), ".format(collected,collect_max,len(df),len(seen_addresses),len(queue),len(new_addresses)))

## Account balances

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from orderedset import OrderedSet
from time import sleep
import sys
import pickle

def get_balance(balance_dict,addresses):
    URL = "https://api.etherscan.io/api"    
    # parameters
    api_key = "8FHTNPR8GQPAEVSRHBVDZ4TWEJQCCUB1NX"
    # defining a params dict for the parameters to be sent to the API
    PARAMS = {'module':'account','action':'balancemulti','address':",".join(addresses),'apikey':api_key}
    
    # sending get request and saving the response as response object
    try:
      r = requests.get(url = URL, params = PARAMS)
    except Exception:
      # If connection error, skip this address
      print("[Error] Error connecting to API, skipping address...")
      return pd.DataFrame({"address":[],"balance":[]})
    
    # Parse data into JSON
    data = r.json()

    # Add all balances to balance_dict
    for row in data["result"]:
        balance_dict[row["account"]] = int(row["balance"])

In [ ]:
balances = dict()

In [2]:
in_file = open("snapshots/jan_17.txt",'r')

seen_addresses = set()
addresses = []

printed_k = 0

while True:
    line = in_file.readline()
    if not line:
        break
    s = line.split(",")
    try:
        from_addr = s[0]
        if from_addr not in balances.keys():
            addresses.append(from_addr)
    except Exception:
        pass
    try:
        to_addr = s[1]
        if to_addr not in balances.keys():
            addresses.append(to_addr)
    except Exception:
        pass
    
    if len(addresses) == 20:
        get_balance(balances,addresses)
        addresses = []
    elif len(addresses) > 20:
        temp = addresses.pop()
        get_balance(balances,addresses)
        addresses = [temp]
    
    if len(balances)-printed_k > 5000:
        print("Now {} addresses".format(len(balances)))
        printed_k = len(balances)

pickle_dump(balances,open("balances.pckl",'wb'))
in_file.close()

Now 5007 addresses
Now 10020 addresses
Now 15034 addresses
Now 20054 addresses
Now 25064 addresses
Now 30078 addresses
Now 35088 addresses
Now 40096 addresses
Now 45115 addresses


IndexError: list index out of range

KeyError: 0